In [1]:
# initial imports

print('ITPS Data Analysis Tool Beta v0.1')
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:.2f}'.format
import numpy as np
import sys
from datetime import datetime
import time

from collections import OrderedDict

from bqplot import *
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from bqplot.market_map import MarketMap
from traitlets import link

from ipywidgets import ToggleButtons, VBox, HBox, HTML, Box
from ipywidgets import widgets, link
from ipywidgets import Label, Layout, Style
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML, clear_output, Javascript
import os

# make display 100% of the screen
display(HTML("<style>.container { width:100% !important; }</style>"))

from FTAT_imports import *

time_slices_db = {} #this is the time slices data base, TP#, Description, Start, Stop (indexes)
selected_slice = [] #this is a global variable to store the currently selected slice

## This is for notebooks that do not have code folding:
#cells_visable = True
#def toggle_code_cells(btn):
#    global cells_visable
#    if cells_visable:
#        display(Javascript("$('div.input').hide();"))
#        btn.description = "Show Code Cells"
#    else:
#        display(Javascript("$('div.input').show();"))
#        btn.description = "Hide Code Cells"
#    cells_visable = not cells_visable
    

#toggle_btn = widgets.Button(description="Hide Code Cells")
#toggle_btn.on_click(toggle_code_cells)

#display(toggle_btn)

ITPS Data Analysis Tool Beta v0.1


In [ ]:
# File Selection
print('ITPS IADS Data Reader')
print('Select the CLEANNED data file')
print('If you do not have the CLEANNED file, change the code in the next-next cell to clean it')


f = FileBrowser()
#f.widget()  #
#   <interact with widget, select a path>
# in a separate cell:

In [ ]:
# Selected Data Reading
print('Reading data from {}'.format(f.path))
print('wait for the asterisk on the left to become a number...')
filepath='./IadsDataExport_clean.csv' #
raw_data=pd.read_csv(filepath, encoding='latin1', low_memory=False) #
#raw_data=pd.read_csv(f.path, encoding='latin1', low_memory=False) ##
print('Done.')

In [ ]:
# clean up time from IADS, subsample for fast graphing


## FOR DIRTY DATA, UNCOMMENT NEXT 3 LINES (DIRTY DATA) AND COMMENT OUT THE OTHER 2 LINES (CLEAN DATA)
#print('Cleaning up the data...')  #DIRTY DATA
#raw_data.fillna(value=0, inplace=True)  #DIRTY DATA
#raw_data['Time'] = (raw_data['Time'].str.slice_replace(0,4,''))  #DIRTY DATA

raw_data['Time'] = pd.to_datetime(raw_data['Time'])   #CLEAN DATA
raw_data = raw_data.set_index(['Time'])   #CLEAN DATA
###
print('Imported data from: {} to: {}, with {} records'.format(raw_data.index[0], raw_data.index[-1], len(raw_data.index) ))

# Let's subsample to speed up the graphing process
desired_sub_sample_rate = 5 #Hz
counter = 0
start_sample = raw_data.index[0]
next_sample = start_sample
while (next_sample - start_sample <= np.timedelta64(1, 's')):
    counter = counter + 1
    next_sample = raw_data.index[counter]
if desired_sub_sample_rate <= counter:
    sub_sample_rate = int(counter / desired_sub_sample_rate)
else:
    sub_sample_rate = int(counter)
    desired_sub_sample_rate = counter
    
sub_sampled_data = raw_data[::sub_sample_rate]

print('Original data at {} Hz'.format((counter-1)))
print('Data subsampled to {} Hz to speed up graphing. Reduced to {} records'.format(desired_sub_sample_rate, len(sub_sampled_data.index)))


In [ ]:
# Slice Map and Definitions
slicemap = ParameterMap(sub_sampled_data, 'MediumSeaGreen', 'Parameter Map')
slicemap.map.on_hover(slicemap.hover_handler)
preview_button = widgets.ToggleButton(
    value=False,
    description='Preview Data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Activate to enable data preview'
)
def toggle_data_preview(b):
    if b.new:
        slicemap.map.enable_hover = True
    else:
        slicemap.map.enable_hover = False

preview_button.observe(toggle_data_preview, "value")

In [ ]:
# Slice Graph Definitions
print('Preparing Graph Parameters ... may take a while ... patience is a virtue.')
# data
current_plot_data = sub_sampled_data
x_main_graph_data = current_plot_data.index.values
y_main_graph_data = current_plot_data.index.values

slice_plot = ZoomPlot(current_plot_data.index.values, current_plot_data.index.values)
detail_plot = ZoomPlot(current_plot_data.index.values, current_plot_data.index.values)
detail_plot_stats = widgets.HTML(
    value="Empty <b>Empty</b>",
    placeholder='Stats',
    description='Stats',
)
detail_plot_stats.value = 'Empty'

In [ ]:
# Slice Logics
deb_output = Label()

def selected_index_changed(change):
    deb_output.value = str(change.new)
    if len(slicemap.map.selected) != 0:
        global current_plot_data
        selected_parameters_plot_data = np.transpose(sub_sampled_data.as_matrix(slicemap.map.selected))
        current_plot_data = sub_sampled_data[slicemap.map.selected]
        slice_plot.line.y = selected_parameters_plot_data
        slice_plot.fig.title = str(slicemap.map.selected)
        detail_plot.line.y = selected_parameters_plot_data
        detail_plot.fig.title = str(slicemap.map.selected) + ' - Zoomed'

slicemap.map.observe(selected_index_changed, 'selected')

slicebox = sliceSelectDialog(current_plot_data)
#HBox(children=slicebox.slice_box_items)

In [ ]:
# Plot Button Logic
def on_plot_button_clicked(b):
    global current_plot_data, selected_slice, sliced_data
    slice_start_time = current_plot_data.index[0]
    slice_start_time = slice_start_time.replace(hour=slicebox.start_hour_box.value, 
                                                minute=slicebox.start_minute_box.value,
                                                second=slicebox.start_second_box.value)
    if slice_start_time < current_plot_data.index[0]:
        slice_start_time = current_plot_data.index[0]
    slice_end_time = current_plot_data.index[0]
    slice_end_time = slice_start_time.replace(hour=slicebox.end_hour_box.value, 
                                              minute=slicebox.end_minute_box.value,
                                              second=slicebox.end_second_box.value)
    if (slice_end_time > current_plot_data.index[-1]) or (slice_end_time <= slice_start_time) :
        slice_end_time = current_plot_data.index[-1]
    #the reason I store the selected_slice is growth capability
    #in the future, I want to maybe use these times to slice a movie
    #therefore I cannot store/lookup the index of the data. Need times.
    

    #this  of the code USES the FULLY sampled data for the zoomed graph
    #for now, I will stick with the subsampled data and use the full set when analyzing only
    '''
    slice_start_index = raw_data.index.searchsorted(slice_start_time)
    slice_end_index = raw_data.index.searchsorted(slice_end_time)
    sliced_data = raw_data[current_plot_data.columns.values].iloc[slice_start_index:slice_end_index]
    x_fig2_graph_data = sliced_data.index.values
    y_fig2_graph_data = np.transpose(sliced_data)
    selected_slice = [sliced_data.index.values[0], sliced_data.index.values[-1]]
    '''
    
    #this version of the code does *NOT* use the fully sampled data for the zoomed graph
    slice_start_index = current_plot_data.index.searchsorted(slice_start_time)
    slice_end_index = current_plot_data.index.searchsorted(slice_end_time)
    sliced_data = current_plot_data.iloc[slice_start_index:slice_end_index]
    selected_slice = [sliced_data.index.values[0], sliced_data.index.values[-1]]

    detail_plot.line.x = sliced_data.index.values
    detail_plot.line.y = np.transpose(sliced_data)
    
    detail_plot_stats.value = sliced_data.describe().to_html()

plot_button = widgets.Button(description='Plot Slice Below')
plot_button.on_click(on_plot_button_clicked)


In [ ]:
# Graph Generation

print('TIME SLICES SELECTION')
print('1. Fine tune Time Slice (TS) with H:M:S boxes')
print('2. Select Plot Slice Below')
print('3. Set TP# and brief description')
print('4. Hit Save TSlice')
print('5. Do next slice')
# Save Slice Button and Logic

# time slices are saved to a dictionary for later analysis
TP_number_box = widgets.FloatText(
    value=0.0,
    description='TP#:',
    disabled=False,
    layout=Layout(width='130px')
)

TP_desc_box = widgets.Text(
    value='',
    placeholder='Type something',
    description='TP Title:',
    disabled=False
)
TS_message = Label()

##analysis widgets moved up to cope with interaction
data_slice_selec = DataSliceSelect(time_slices_db)

#data_slice_selec_link = link((TP_saved_dd, 'options'), (data_slice_selec.analysisTPdd, 'options'))
###

def on_saveTS_button_clicked(b):
    global time_slices_db
    if str(TP_number_box.value) in time_slices_db:
        TS_message.value = 'TP# {} already in db, choose a different one'.format(TP_number_box.value)
    else:
        TS_message.value = 'added TP {}'.format(TP_number_box.value)
        if detail_plot.xs.min:  # if the scale is None, go to ELSE!
            time_slices_db[str(TP_number_box.value)] = [TP_desc_box.value, np.datetime64(detail_plot.xs.min)
                                                        , np.datetime64(detail_plot.xs.max)]
        else:
            time_slices_db[str(TP_number_box.value)] = [TP_desc_box.value, selected_slice[0], selected_slice[-1]]
        TP_saved_dd.options = list(dict.keys(time_slices_db))
        data_slice_selec.analysisTPdd.options = list(dict.keys(time_slices_db))
        
def on_delTS_button_clicked(b):
    global time_slices_db
    if TP_saved_dd.value in time_slices_db:
        TS_message.value = 'Deleted'
        del time_slices_db[TP_saved_dd.value]
        TP_saved_dd.options = list(dict.keys(time_slices_db))
    else:
        TS_message.value = 'trying to delete nothing?'


TP_saved_dd = widgets.Dropdown(
    options=time_slices_db,
    description='TP#:',
    disabled=False,
)

def on_TP_saved_dd_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        #print("changed to %s" % change['new'])
        TS_message.value = ''

TP_saved_dd.observe(on_TP_saved_dd_change)
    
    
saveTS_button = widgets.Button(description='Save TSlice')
saveTS_button.on_click(on_saveTS_button_clicked)
delTS_button = widgets.Button(description='Delete TSlice')
delTS_button.on_click(on_delTS_button_clicked)


######http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
# actual dashboard

# list of items to be displayed
slicer_form_items = [
    VBox([deb_output, preview_button, slicemap.map]),
    VBox([slice_plot.fig, HBox([slice_plot.zoom_interacts,slice_plot.reset_zoom_button])]),
    HBox(children=slicebox.slice_box_items),
    Box([plot_button]),
    HBox([detail_plot.fig, detail_plot_stats]),
    HBox([detail_plot.zoom_interacts,detail_plot.reset_zoom_button]),
    VBox([TS_message, HBox([saveTS_button, TP_number_box, TP_desc_box, TP_saved_dd, delTS_button])])
]

# actual form definition
slicer_form = Box(slicer_form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='100%'
))
slicer_form

In [ ]:
# Analysis Map Definitions

print('Generating Analysis Canvas...')
analysismap = ParameterMap(sub_sampled_data, 'Orange', 'Parameter Map')

deb_output = Label()

def selected_index_changed(change):
    if len(analysismap.map.selected) != 0:
        global current_plot_data
        selected_param_plot_data = np.transpose(sub_sampled_data.as_matrix(analysismap.map.selected))
        current_plot_data = sub_sampled_data[analysismap.map.selected]
        analysis_plot.line.y = selected_param_plot_data
        analysis_plot.y_data = selected_param_plot_data  ##
        analysis_plot.fig.title = str(analysismap.map.selected)
        analysis_plot.xs.min = None
        analysis_plot.xs.max = None

analysismap.map.observe(selected_index_changed, 'selected')
if analysismap.map.selected is not None:
    current_plot_data = sub_sampled_data[analysismap.map.selected]
else:
    current_plot_data = sub_sampled_data
analysis_plot = AnalysisPlot(current_plot_data.index.values, current_plot_data.index.values, time_slices_db)


poly_order = 1


def on_analysis_poly_order_dd_change(change):
    global poly_order, current_plot_data
    if change['type'] == 'change' and change['name'] == 'value' and len(analysismap.map.selected) != 0:
        poly_order = change['new']
        #print(poly_order)
        analysis_plot.update_plot(current_plot_data, analysismap.map.selected, analysis_plot.xs.min, analysis_plot.xs.max, poly_order)
        #current_plot_data, parameter_list, slice_start, slice_end, poly_degree
            
data_slice_selec.analysisPolyOrderdd.observe(on_analysis_poly_order_dd_change)


def on_analysis_TP_dd_change(change):
    global poly_order, current_plot_data, time_slices_db
    if change['type'] == 'change' and change['name'] == 'value' and len(analysismap.map.selected) != 0:
        data_slice_selec.analysisTPDescBox.value = change['new'][0]
        data_slice_selec.analysisPolyOrderdd.disabled=False
        analysis_plot.x_data_slice_min = time_slices_db[change['new']][1]
        analysis_plot.x_data_slice_max = time_slices_db[change['new']][2]
        analysis_plot.update_plot(current_plot_data, analysismap.map.selected, time_slices_db[change['new']][1],
                                  time_slices_db[change['new']][2], poly_order)
        
data_slice_selec.analysisTPdd.observe(on_analysis_TP_dd_change)


analysis_plot_zoom_slider = SimpleZoomSlider(analysis_plot)



def on_analysis_zoom_slider_change(change):
    analysis_plot_zoom_slider.updateScale(analysis_plot)
    analysis_plot.update_plot(current_plot_data, analysismap.map.selected, 
                              np.datetime64(analysis_plot.xs.min),
                              np.datetime64(analysis_plot.xs.max),
                              poly_order)

analysis_plot_zoom_slider.zoom_slider.observe(on_analysis_zoom_slider_change, names='value')

# slice trim to zoomed figure logic
def slice_trim(tp_dict, key_to_find, slice_start, slice_end):
    for key in tp_dict.keys():
        if key == key_to_find:
            current_value = tp_dict[key]
            current_value[1] = slice_start
            current_value[2] = slice_end
            tp_dict[key] = current_value


def on_slice_trim_button_clicked(b):
    slice_trim(time_slices_db, data_slice_selec.analysisTPdd.value, np.datetime64(analysis_plot.xs.min), np.datetime64(analysis_plot.xs.max))
    
slice_trim_button = widgets.Button(description='Trim Current Slice')
slice_trim_button.on_click(on_slice_trim_button_clicked)

In [ ]:
# Time Slices Analysis Section

print('TS ANALYSIS')
print('1. select parameter')
print('2. select saved TS')
analysis_form_items = [
    analysismap.map,
    HBox([data_slice_selec.analysisTPdd, data_slice_selec.analysisTPDescBox, data_slice_selec.analysisPolyOrderdd]),
    VBox([HBox([analysis_plot.fig, analysis_plot.fit_statistics]), analysis_plot.plot_toolbar]),
    analysis_plot_zoom_slider.zoom_slider,
    slice_trim_button
]

# actual form definition
analysis_form = Box(analysis_form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='100%'
))
analysis_form

In [ ]:
# Slices Save to Disk Section

# slice trim to zoomed figure logic
def save_slices(tp_dict):
    for key in tp_dict.keys():
        current_value = tp_dict[key]
        filename = '~/work/' + 'TP_' +  str(key) + '_' + current_value[0] + '.csv'
        #filename = f.path.split('.')[0] + '_TP_' +  str(key) + '_' + current_value[0] + '.csv'
        #think where we can save this so the user can retrieve later...
        #remember we are in a container.
        slice_start = current_value[1]
        slice_end = current_value[2]
        sliced_data = raw_data.iloc[(raw_data.index >= slice_start) & 
                                           (raw_data.index <= slice_end)]
        sliced_data.to_csv(filename)
    _ = !rm TP_zipped.zip
    !zip TP_zipped.zip TP*.csv
    !rm TP*.csv
    print('')
    print('Your slices were saved to TP_zipped.zip')
    print('Use the top menu to download to your PC')
        


def on_save_slices_button_clicked(b):
    save_slices(time_slices_db)
    
save_slices_button = widgets.Button(description='Save Slices to Disk')
save_slices_button.on_click(on_save_slices_button_clicked)
save_slices_button

In [ ]:
# Exploratory Section Strip Charts

##TODO CLEAN UP BUTTON THAT PLOTS WITHOUT REFRESH AFTER HMI DICUSSION

print('This is an exploratory section')
print('1. Select parameters for Strip Chart')
print('2. Press Plot Strip Chart Below')
print('3. The TS is the one selected above')

strip_chart_map =  ParameterMap(sub_sampled_data, 'Blue', 'Parameter Map')

    
strip_chart_button = widgets.Button(description='Plot Strip Chart Below')

strip_chart = StripChart(sub_sampled_data, strip_chart_map, analysis_plot)


def on_strip_chart_button_clicked(b):
    if len(strip_chart_map.map.selected) != 0:
        #1 select time slice of current plot above
        slice_start = np.datetime64(analysis_plot.xs.min)
        slice_end = np.datetime64(analysis_plot.xs.max)
        

        slice_start_index = sub_sampled_data.index.searchsorted(slice_start)
        initial_value = sub_sampled_data.index[slice_start_index].timestamp()


        strip_chart_x_data = sub_sampled_data.iloc[(sub_sampled_data.index >= slice_start) & 
                                       (sub_sampled_data.index <= slice_end)].index


        #3 for each parameter, create a zoom plot
        plotlist = []
        counter = 0
        for selection in strip_chart_map.map.selected:
            strip_chart_y_data = sub_sampled_data.iloc[(sub_sampled_data.index >= slice_start) & 
                                       (sub_sampled_data.index <= slice_end)][selection].values
            plotlist.append(LinePlot(strip_chart_x_data, strip_chart_y_data))
            plotlist[counter].fig.title = data_slice_selec.analysisTPdd.value + ' / ' + selection
            counter += 1


        # Time Slices Analysis Section
        test_analysis_form_items1 = []
        for plots in plotlist:
            plots.xs.min = None
            plots.xs.max = None
            plots.ys.min = None
            plots.ys.max = None
            plots.xax.tick_format = '%M:%S.%L'
            test_analysis_form_items1.append(plots.fig)
        test_analysis_form_items = [
            VBox(test_analysis_form_items1)
        ]

        # actual form definition
        test_analysis_form = Box(test_analysis_form_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px',
            align_items='stretch',
            width='100%'
        ))
        display(test_analysis_form)

strip_chart_button.on_click(on_strip_chart_button_clicked)

#VBox([strip_chart_map.map, strip_chart_button])
VBox([strip_chart_map.map, strip_chart.widget()])


In [ ]:
# Exploratory Section X-Plot

print('This is an exploratory section')
print('1. Select 2 parameters for Cross Plot')
print('2. Press Cross Plot Chart Below')
print('3. The TS is the one selected above')
xplot_map =  ParameterMap(sub_sampled_data, 'Blue', 'Parameter Map')

    
xplot_button = widgets.Button(description='Cross Plot Chart Below')


def on_xplot_button_clicked(b):
    if len(xplot_map.map.selected) > 1:
        #1 select time slice of current plot above
        slice_start = np.datetime64(analysis_plot.xs.min)
        slice_end = np.datetime64(analysis_plot.xs.max)
        

        slice_start_index = sub_sampled_data.index.searchsorted(slice_start)
        initial_value = sub_sampled_data.index[slice_start_index].timestamp()



        #3 for each parameter, create a zoom plot

        xplot_x_data = sub_sampled_data.iloc[(sub_sampled_data.index >= slice_start) & 
                                       (sub_sampled_data.index <= slice_end)][xplot_map.map.selected[0]].values
        
        xplot_y_data = sub_sampled_data.iloc[(sub_sampled_data.index >= slice_start) & 
                                       (sub_sampled_data.index <= slice_end)][xplot_map.map.selected[1]].values
        xplot = CrossPlot(xplot_x_data, xplot_y_data)
        xplot.fig.title = data_slice_selec.analysisTPdd.value + ' / ' + xplot_map.map.selected[0] + ' vs ' + xplot_map.map.selected[1]
        xplot.xax.label = xplot_map.map.selected[0]
        xplot.yax.label = xplot_map.map.selected[1]
        print(xplot.xax.label)

        display(xplot.fig)

xplot_button.on_click(on_xplot_button_clicked)

VBox([xplot_map.map, xplot_button])


In [ ]:
# Exploratory Section - other data analysis
# For example, to calculate total velocity from GPS (SBG) data:
sub_sampled_data['SBG_TOT_Vel'] = (sub_sampled_data['SBG_NVel']**2+sub_sampled_data['SBG_EVel']**2+sub_sampled_data['SBG_DVel']**2)**(0.5)

# To calculate the velocity derivative
sub_sampled_data['DSBG_TOT_Vel'] = sub_sampled_data['SBG_TOT_Vel'].diff() / sub_sampled_data.index.to_series().diff().dt.total_seconds()

# To calculate the total aircraf energy, we can:
# 1. define the aircraft mass at start of test
AC_initial_mass = 6900 #lbs
# 2. define conversion factor from gal/h to kg/h,  gravity constants
gal2kg = 1/0.17 * 0.453592
kt2ms = 0.514444
g = 9.8
ft2m = 0.3048
# 3. integrate fuel flow to get used fuel
sub_sampled_data['fuel_used'] = (sub_sampled_data['T28001_EU'].cumsum() + sub_sampled_data['T28002_EU'].cumsum()) * (sub_sampled_data.index.to_series().diff().dt.total_seconds()/60/60)
# 4. calculate the instantaneous weight by integrating fuel flow:
# initial mass - (sum of fuel flows) * delta t
sub_sampled_data['AC_mass'] = AC_initial_mass - (sub_sampled_data['fuel_used']) * gal2kg
# 5. calculate the energy
sub_sampled_data['Total_Energy'] = 0.5 * sub_sampled_data['AC_mass'] * (sub_sampled_data['SBG_TOT_Vel']*kt2ms)**2 + sub_sampled_data['AC_mass'] * g * sub_sampled_data['Altitude'] #J


In [ ]:
# Exploratory Section - Additional Plots

# We can now plot what we calculated:
# 1.define the time slice we want
slice_start = np.datetime64('2018-03-27 14:46:01')
slice_end = np.datetime64('2018-03-27 14:47:00')

# 2a.slice the x data
my_x_data = sub_sampled_data.iloc[(sub_sampled_data.index >= slice_start) & 
                                       (sub_sampled_data.index <= slice_end)].index
# 2b.choose&slice the y data
my_y_data = sub_sampled_data.iloc[(sub_sampled_data.index >= slice_start) & 
                                       (sub_sampled_data.index <= slice_end)]['Total_Energy'].values
# 3.create the plit
my_graph = LinePlot(my_x_data, my_y_data)

# 4.display it
my_graph.fig